## Importing Packages

In [52]:
import requests
import json
from datetime import datetime
import pandas as pd

from sqlalchemy import create_engine # database connection

# DataSet retrival Procedure

## Definition

In [ ]:
BASE_URL = "https://earthquake.usgs.gov/fdsnws/event/1/query"

all_earthquakes = []

## Data fetching

In [ ]:
all_earthquakes = []

for year in range(2020, 2026):
    for month in range(1, 13):

        start_date = datetime(year, month, 1)

        if month == 12:
            end_date = datetime(year + 1, 1, 1)
        else:
            end_date = datetime(year, month + 1, 1)

        starttime = start_date.strftime("%Y-%m-%d")
        endtime = end_date.strftime("%Y-%m-%d")

        print(f"Fetching data for {starttime} to {endtime}")

        params = {
            "format": "geojson",
            "starttime": starttime,
            "endtime": endtime,
            "minmagnitude": 4.5,
            "orderby": "time",
            "limit": 10000
        }

        try:
            response = requests.get(BASE_URL, params=params, timeout=30)

            if response.status_code == 200:
                data = response.json()
                count = len(data["features"])
                print(f"  Records: {count}")
                all_earthquakes.extend(data["features"])
            else:
                print(f"  FAILED: {response.status_code}")

        except requests.exceptions.RequestException as e:
            print("  ERROR:", e)

# Save JSON output
output_file = "earthquake_2020_2025_monthwise.json"
with open(output_file, "w") as f:
    json.dump(all_earthquakes, f, indent=4)

print("Data saved successfully in JSON format")


Fetching data for 2020-01-01 to 2020-02-01
  Records: 657
Fetching data for 2020-02-01 to 2020-03-01
  Records: 522
Fetching data for 2020-03-01 to 2020-04-01
  Records: 497
Fetching data for 2020-04-01 to 2020-05-01
  Records: 456
Fetching data for 2020-05-01 to 2020-06-01
  Records: 559
Fetching data for 2020-06-01 to 2020-07-01
  Records: 587
Fetching data for 2020-07-01 to 2020-08-01
  Records: 552
Fetching data for 2020-08-01 to 2020-09-01
  Records: 529
Fetching data for 2020-09-01 to 2020-10-01
  Records: 557
Fetching data for 2020-10-01 to 2020-11-01
  Records: 596
Fetching data for 2020-11-01 to 2020-12-01
  Records: 471
Fetching data for 2020-12-01 to 2021-01-01
  Records: 528
Fetching data for 2021-01-01 to 2021-02-01
  Records: 576
Fetching data for 2021-02-01 to 2021-03-01
  Records: 849
Fetching data for 2021-03-01 to 2021-04-01
  Records: 1262
Fetching data for 2021-04-01 to 2021-05-01
  Records: 599
Fetching data for 2021-05-01 to 2021-06-01
  Records: 626
Fetching data

## Data Reading from Json

In [ ]:
with open ("earthquake_2020_2025_monthwise.json","r") as file:
    data = json.load(file)
    print(type(data))
print("Total records:", len(data))
print("Sample record keys:", data[0].keys())

<class 'list'>
Total records: 45840
Sample record keys: dict_keys(['type', 'properties', 'geometry', 'id'])


## Json to dataframe conversion

In [ ]:
records = []

for eq in data:
    prop = eq.get("properties", {})
    geo = eq.get("geometry", {})
    coords = geo.get("coordinates", [None, None, None])

    records.append({
        "id": eq.get("id"),
        "time": prop.get("time"),
        "updated": prop.get("updated"),
        "mag": prop.get("mag"),
        "magType": prop.get("magType"),
        "place": prop.get("place"),
        "status": prop.get("status"),
        "tsunami": prop.get("tsunami"),
        "sig": prop.get("sig"),
        "net": prop.get("net"),
        "nst": prop.get("nst"),
        "dmin": prop.get("dmin"),
        "rms": prop.get("rms"),
        "gap": prop.get("gap"),
        "magError": prop.get("magError"),
        "depthError": prop.get("depthError"),
        "magNst": prop.get("magNst"),
        "locationSource": prop.get("locationSource"),
        "magSource": prop.get("magSource"),
        "types": prop.get("types"),
        "ids": prop.get("ids"),
        "sources": prop.get("sources"),
        "type": prop.get("type"),
        "longitude": coords[0],
        "latitude": coords[1],
        "depth_km": coords[2]
    })

df = pd.DataFrame(records)

df.head()


,id,time,updated,mag,magType,place,status,tsunami,sig,net,...,magNst,locationSource,magSource,types,ids,sources,type,longitude,latitude,depth_km
0,us60007lub,1580513571609,1587247807040,4.7,mwr,"6 km NNE of Sivrice, Turkey",reviewed,0,340,us,...,None,None,None,",moment-tensor,origin,phase-data,",",us60007lub,",",us,",earthquake,39.3382,38.4961,10.00
1,us60007lul,1580511971255,1587247807040,4.7,mb,"21 km ENE of San Bartolo, Peru",reviewed,0,348,us,...,None,None,None,",dyfi,origin,phase-data,",",us60007lul,",",us,",earthquake,-76.6148,-12.2747,75.51
2,us60007lsc,1580508655844,1766428058231,5.0,mww,Banda Sea,reviewed,0,385,us,...,None,None,None,",dyfi,moment-tensor,origin,phase-data,",",us60007lsc,iscgem617252948,",",us,iscgem,",earthquake,129.1523,-6.4065,221.39
3,us60007lp8,1580502868793,1587247806040,4.5,mb,"217 km S of Punta de Burica, Panama",reviewed,0,312,us,...,None,None,None,",origin,phase-data,",",us60007lp8,",",us,",earthquake,-82.5400,6.0924,10.00
4,us60007lia,1580490466302,1766430336284,5.3,mww,"2 km SW of Toride, Japan",reviewed,0,641,us,...,None,None,None,",dyfi,moment-tensor,origin,phase-data,",",us60007lia,iscgem621622096,",",us,iscgem,",earthquake,140.0639,35.8840,64.00


## Exploratary Data Analysis

In [ ]:
df.shape

(45840, 26)

In [ ]:
df.columns

Index(['id', 'time', 'updated', 'mag', 'magType', 'place', 'status', 'tsunami',
       'sig', 'net', 'nst', 'dmin', 'rms', 'gap', 'magError', 'depthError',
       'magNst', 'locationSource', 'magSource', 'types', 'ids', 'sources',
       'type', 'longitude', 'latitude', 'depth_km'],
      dtype='str')

In [ ]:
df.types

0             ,moment-tensor,origin,phase-data,
1                      ,dyfi,origin,phase-data,
2        ,dyfi,moment-tensor,origin,phase-data,
3                           ,origin,phase-data,
4        ,dyfi,moment-tensor,origin,phase-data,
                          ...                  
45835                       ,origin,phase-data,
45836                       ,origin,phase-data,
45837                       ,origin,phase-data,
45838                       ,origin,phase-data,
45839                       ,origin,phase-data,
Name: types, Length: 45840, dtype: str

In [ ]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 45840 entries, 0 to 45839
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              45840 non-null  str           
 1   time            45840 non-null  datetime64[ms]
 2   updated         45840 non-null  datetime64[ms]
 3   mag             45840 non-null  float64       
 4   magType         45840 non-null  str           
 5   place           45815 non-null  str           
 6   status          45840 non-null  str           
 7   tsunami         45840 non-null  int64         
 8   sig             45840 non-null  int64         
 9   net             45840 non-null  str           
 10  nst             27686 non-null  float64       
 11  dmin            45602 non-null  float64       
 12  rms             45807 non-null  float64       
 13  gap             45618 non-null  float64       
 14  magError        0 non-null      object        
 15  depthError   

In [ ]:
print("Total null values:", df.isnull().sum().sum())

Total null values: 0


In [ ]:
df.describe()

,time,updated,mag,tsunami,sig,nst,dmin,rms,gap,longitude,latitude,depth_km
count,45840,45840,45840.000000,45840.000000,45840.000000,27686.000000,45602.000000,45807.000000,45618.000000,45840.000000,45840.000000,45840.000000
mean,2023-01-10 11:27:26.140000,2023-08-19 10:06:07.638000,4.802572,0.012260,361.843717,69.269125,4.446062,0.733537,91.729052,36.126689,0.045631,57.902189
min,2020-01-01 00:28:20.289000,2020-03-21 17:13:04.040000,4.500000,0.000000,312.000000,0.000000,0.000000,0.000000,8.000000,-179.999700,-73.220400,-0.570000
25%,2021-07-27 14:19:49.865000,2022-03-01 16:39:22.040000,4.500000,0.000000,312.000000,34.000000,1.363000,0.570000,59.000000,-70.266925,-22.451875,10.000000
50%,2022-12-15 05:25:37.699000,2023-10-07 21:39:01.040000,4.700000,0.000000,340.000000,56.000000,2.544000,0.710000,86.000000,94.831450,-3.286300,10.000000
75%,2024-07-06 10:23:22.132000,2025-05-02 16:26:16.040000,4.900000,0.000000,370.000000,90.000000,5.140750,0.870000,120.000000,141.951350,24.782400,55.225250
max,2025-12-31 23:04:28.213000,2026-02-03 09:12:39.803000,8.800000,1.000000,2910.000000,619.000000,62.558000,2.820000,348.000000,179.998400,87.386000,669.556000
std,NaN,NaN,0.368080,0.110045,88.570789,49.830940,5.582670,0.221201,42.834830,122.437876,31.576477,106.914575


In [ ]:
df.isnull().sum()
print(df.isnull().sum())

id                    0
time                  0
updated               0
mag                   0
magType               0
place                25
status                0
tsunami               0
sig                   0
net                   0
nst               18154
dmin                238
rms                  33
gap                 222
magError          45840
depthError        45840
magNst            45840
locationSource    45840
magSource         45840
types                 0
ids                   0
sources               0
type                  0
longitude             0
latitude              0
depth_km              0
dtype: int64


## Data Conversions

In [ ]:
# Convert epoch milliseconds to datetime
df["time"] = pd.to_datetime(df["time"], unit="ms")
df["updated"] = pd.to_datetime(df["updated"], unit="ms")
print(df[["time", "updated"]].head())

                     time                 updated
0 2020-01-31 23:32:51.609 2020-04-18 22:10:07.040
1 2020-01-31 23:06:11.255 2020-04-18 22:10:07.040
2 2020-01-31 22:10:55.844 2025-12-22 18:27:38.231
3 2020-01-31 20:34:28.793 2020-04-18 22:10:06.040
4 2020-01-31 17:07:46.302 2025-12-22 19:05:36.284


## Data Cleaning

In [ ]:
numeric_cols = [
    "mag", "depth_km", "nst", "dmin", "rms", "gap",
    "magError", "depthError", "magNst", "sig",
    "latitude", "longitude"
]

text_cols = [
    "place", "magType", "status", "type", "net",
    "sources", "ids", "types", "locationSource", "magSource"
]


df[numeric_cols] = df[numeric_cols].fillna(0)
df[text_cols] = df[text_cols].fillna("unknown")

df.head()

,id,time,updated,mag,magType,place,status,tsunami,sig,net,...,magNst,locationSource,magSource,types,ids,sources,type,longitude,latitude,depth_km
0,us60007lub,2020-01-31 23:32:51.609,2020-04-18 22:10:07.040,4.7,mwr,"6 km NNE of Sivrice, Turkey",reviewed,0,340,us,...,0,unknown,unknown,",moment-tensor,origin,phase-data,",",us60007lub,",",us,",earthquake,39.3382,38.4961,10.00
1,us60007lul,2020-01-31 23:06:11.255,2020-04-18 22:10:07.040,4.7,mb,"21 km ENE of San Bartolo, Peru",reviewed,0,348,us,...,0,unknown,unknown,",dyfi,origin,phase-data,",",us60007lul,",",us,",earthquake,-76.6148,-12.2747,75.51
2,us60007lsc,2020-01-31 22:10:55.844,2025-12-22 18:27:38.231,5.0,mww,Banda Sea,reviewed,0,385,us,...,0,unknown,unknown,",dyfi,moment-tensor,origin,phase-data,",",us60007lsc,iscgem617252948,",",us,iscgem,",earthquake,129.1523,-6.4065,221.39
3,us60007lp8,2020-01-31 20:34:28.793,2020-04-18 22:10:06.040,4.5,mb,"217 km S of Punta de Burica, Panama",reviewed,0,312,us,...,0,unknown,unknown,",origin,phase-data,",",us60007lp8,",",us,",earthquake,-82.5400,6.0924,10.00
4,us60007lia,2020-01-31 17:07:46.302,2025-12-22 19:05:36.284,5.3,mww,"2 km SW of Toride, Japan",reviewed,0,641,us,...,0,unknown,unknown,",dyfi,moment-tensor,origin,phase-data,",",us60007lia,iscgem621622096,",",us,iscgem,",earthquake,140.0639,35.8840,64.00


In [ ]:

# for col in numeric_cols:
#     df[col] = pd.to_numeric(df[col], errors="coerce")
# df.head(3)

In [ ]:
df.columns = df.columns.str.lower().str.replace(" ", "_")
df.head(2)

,id,time,updated,mag,magtype,place,status,tsunami,sig,net,...,magnst,locationsource,magsource,types,ids,sources,type,longitude,latitude,depth_km
0,us60007lub,2020-01-31 23:32:51.609,2020-04-18 22:10:07.040,4.7,mwr,"6 km NNE of Sivrice, Turkey",reviewed,0,340,us,...,0,unknown,unknown,",moment-tensor,origin,phase-data,",",us60007lub,",",us,",earthquake,39.3382,38.4961,10.00
1,us60007lul,2020-01-31 23:06:11.255,2020-04-18 22:10:07.040,4.7,mb,"21 km ENE of San Bartolo, Peru",reviewed,0,348,us,...,0,unknown,unknown,",dyfi,origin,phase-data,",",us60007lul,",",us,",earthquake,-76.6148,-12.2747,75.51


### Regex

In [ ]:
df["country"] = df["place"].str.extract(r",\s*([^,]+)$")
df["country"]

0             Turkey
1               Peru
2                NaN
3             Panama
4              Japan
            ...     
45835     Guadeloupe
45836            NaN
45837          Tonga
45838    Philippines
45839          Japan
Name: country, Length: 45840, dtype: str

## Derived columns

### Year, month, day and day of week

In [ ]:
df['time'].dtype

dtype('<M8[ms]')

In [ ]:
df["year"] = df["time"].dt.year
df["year"]

0        2020
1        2020
2        2020
3        2020
4        2020
         ... 
45835    2025
45836    2025
45837    2025
45838    2025
45839    2025
Name: year, Length: 45840, dtype: int32

In [ ]:
df["month"] = df["time"].dt.month
df["month"]

0         1
1         1
2         1
3         1
4         1
         ..
45835    12
45836    12
45837    12
45838    12
45839    12
Name: month, Length: 45840, dtype: int32

In [ ]:
df["day"] = df["time"].dt.day
df["day"]

0        31
1        31
2        31
3        31
4        31
         ..
45835     1
45836     1
45837     1
45838     1
45839     1
Name: day, Length: 45840, dtype: int32

In [ ]:
df["day_of_week"] = df["time"].dt.day_name()
df["day_of_week"]

0        Friday
1        Friday
2        Friday
3        Friday
4        Friday
          ...  
45835    Monday
45836    Monday
45837    Monday
45838    Monday
45839    Monday
Name: day_of_week, Length: 45840, dtype: str

In [ ]:
df[["time", "year", "month", "day", "day_of_week"]].head()

,time,year,month,day,day_of_week
0,2020-01-31 23:32:51.609,2020,1,31,Friday
1,2020-01-31 23:06:11.255,2020,1,31,Friday
2,2020-01-31 22:10:55.844,2020,1,31,Friday
3,2020-01-31 20:34:28.793,2020,1,31,Friday
4,2020-01-31 17:07:46.302,2020,1,31,Friday


### Depth Flag

In [ ]:
df["depth_flag"] = df["depth_km"].apply(
    lambda x: "shallow" if x < 70 else "deep"
)
df["depth_flag"].head()

0    shallow
1       deep
2       deep
3    shallow
4    shallow
Name: depth_flag, dtype: str

### Mag flag

In [ ]:
df["magflag"] = df["mag"].apply(
    lambda x: "destructive" if x >= 6.0 else "strong"
)
df["magflag"]

0        strong
1        strong
2        strong
3        strong
4        strong
          ...  
45835    strong
45836    strong
45837    strong
45838    strong
45839    strong
Name: magflag, Length: 45840, dtype: str

# Store data to MySQL

In [53]:
username = "root"
password = "12345"          # your MySQL password
host = "localhost"
port = 3306                 # INTEGER, not string
database = "earthquake_db"

engine = create_engine(
    f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}"
)

print("MySQL connection created successfully")


MySQL connection created successfully


In [55]:
df = df.rename(columns={
    "mag": "magnitude",
    "magtype": "magnitude_type",
    "nst": "station_count",
    "dmin": "min_station_distance",
    "rms": "rms_error",
    "gap": "azimuthal_gap",
    "sig": "significance",
    "net": "network",
    "type": "event_type",
    "magerror": "magnitude_error",
    "deptherror": "depth_error",
    "magnst": "magnitude_station_count",
    "locationsource": "location_source",
    "magsource": "magnitude_source"
})

In [107]:
df.to_sql(
    name="earthquakes",
    con=engine,
    if_exists="append",
    index=False
)
print("Data inserted into MySQL successfully")

Data inserted into MySQL successfully


45840

# Insights using SQL

## Magnitude & Depth

In [ ]:
query = "SELECT * FROM earthquakes"

df_mysql = pd.read_sql(query, con=engine)

In [ ]:
df_mysql.head()

,id,time,updated,magnitude,magnitude_type,place,status,tsunami,significance,network,...,event_type,longitude,latitude,depth_km,depth_flag,magflag,year,month,day,day_of_week
0,us60007lub,2020-01-31 23:32:52,2020-04-18 22:10:07,4.7,mwr,"6 km NNE of Sivrice, Turkey",reviewed,0,340,us,...,earthquake,39.3382,38.4961,10.00,shallow,strong,2020,1,31,Friday
1,us60007lul,2020-01-31 23:06:11,2020-04-18 22:10:07,4.7,mb,"21 km ENE of San Bartolo, Peru",reviewed,0,348,us,...,earthquake,-76.6148,-12.2747,75.51,deep,strong,2020,1,31,Friday
2,us60007lsc,2020-01-31 22:10:56,2025-12-22 18:27:38,5.0,mww,Banda Sea,reviewed,0,385,us,...,earthquake,129.1523,-6.4065,221.39,deep,strong,2020,1,31,Friday
3,us60007lp8,2020-01-31 20:34:29,2020-04-18 22:10:06,4.5,mb,"217 km S of Punta de Burica, Panama",reviewed,0,312,us,...,earthquake,-82.5400,6.0924,10.00,shallow,strong,2020,1,31,Friday
4,us60007lia,2020-01-31 17:07:46,2025-12-22 19:05:36,5.3,mww,"2 km SW of Toride, Japan",reviewed,0,641,us,...,earthquake,140.0639,35.8840,64.00,shallow,strong,2020,1,31,Friday


### 1. Top 10 strongest earthquakes (by magnitude)


In [ ]:
query = "SELECT id,magnitude,depth_km,place,time FROM earthquakes ORDER BY magnitude DESC LIMIT 10;"
pd.read_sql(query, con=engine)

,id,magnitude,depth_km,place,time
0,us6000qw60,8.8,35.000,"2025 Kamchatka Peninsula, Russia Earthquake",2025-07-29 23:24:52
1,ak0219neiszm,8.2,35.000,"2021 Chignik, Alaska Earthquake",2021-07-29 06:15:49
2,us6000f53e,8.1,22.790,2021 South Sandwich Islands Earthquake,2021-08-12 18:35:17
3,us7000dflf,8.1,28.930,"2021 Kermadec Islands, New Zealand Earthquake",2021-03-04 19:28:33
4,us6000jllz,7.8,10.000,"Pazarcik earthquake, Kahramanmaras earthquake ...",2023-02-06 01:17:34
5,us7000qx2g,7.8,27.000,"140 km E of Petropavlovsk-Kamchatsky, Russia",2025-09-18 18:58:15
6,us7000asvb,7.8,28.000,"2020 Perryville, Alaska Earthquake",2020-07-22 06:12:45
7,us6000dg77,7.7,10.000,southeast of the Loyalty Islands,2021-02-10 13:19:56
8,us60007idc,7.7,14.860,"123 km NNW of Lucea, Jamaica",2020-01-28 19:10:25
9,us6000kd0n,7.7,18.053,southeast of the Loyalty Islands,2023-05-19 02:57:03


### 2.  Top 10 deepest earthquakes (by depth_km)

In [ ]:
query = "SELECT id,depth_km,magnitude,place FROM earthquakes ORDER BY depth_km DESC LIMIT 10;"
pd.read_sql(query, con=engine)

,id,depth_km,magnitude,place
0,us6000rk66,669.556,4.8,"205 km ESE of Levuka, Fiji"
1,us6000dhfx,664.740,4.5,south of the Fiji Islands
2,us7000ingi,660.000,7.0,south of the Fiji Islands
3,us7000inmi,656.429,4.9,south of the Fiji Islands
4,us6000fwrg,654.810,4.5,Fiji region
5,us6000ngkc,653.779,4.5,south of the Fiji Islands
6,us6000kq6n,653.516,4.7,"209 km SE of Levuka, Fiji"
7,us7000inmb,650.921,5.1,south of the Fiji Islands
8,us7000kvfm,650.655,5.0,"100 km NW of Batang, Indonesia"
9,us6000py4l,648.298,5.6,"126 km WNW of Iñapari, Peru"


### 3. Shallow earthquakes (< 50 km) with magnitude > 7.5

In [ ]:
query = "SELECT id,magnitude,depth_km,place,time FROM earthquakes WHERE depth_km<50 AND magnitude>7.5 ORDER BY magnitude DESC"
pd.read_sql(query, con=engine)

,id,magnitude,depth_km,place,time
0,us6000qw60,8.8,35.000,"2025 Kamchatka Peninsula, Russia Earthquake",2025-07-29 23:24:52
1,ak0219neiszm,8.2,35.000,"2021 Chignik, Alaska Earthquake",2021-07-29 06:15:49
2,us7000dflf,8.1,28.930,"2021 Kermadec Islands, New Zealand Earthquake",2021-03-04 19:28:33
3,us6000f53e,8.1,22.790,2021 South Sandwich Islands Earthquake,2021-08-12 18:35:17
4,us7000asvb,7.8,28.000,"2020 Perryville, Alaska Earthquake",2020-07-22 06:12:45
5,us6000jllz,7.8,10.000,"Pazarcik earthquake, Kahramanmaras earthquake ...",2023-02-06 01:17:34
6,us7000qx2g,7.8,27.000,"140 km E of Petropavlovsk-Kamchatsky, Russia",2025-09-18 18:58:15
7,us60007idc,7.7,14.860,"123 km NNW of Lucea, Jamaica",2020-01-28 19:10:25
8,us6000dg77,7.7,10.000,southeast of the Loyalty Islands,2021-02-10 13:19:56
9,us6000kd0n,7.7,18.053,southeast of the Loyalty Islands,2023-05-19 02:57:03


### 4. Average depth per continent

In [ ]:
query = "SELECT CASE WHEN place LIKE '%Asia%' THEN 'Asia' WHEN place LIKE '%Europe%' THEN 'Europe' WHEN place LIKE '%Africa%' THEN 'Africa' WHEN place LIKE '%America%' THEN 'America' WHEN place LIKE '%Australia%' THEN 'Australia' WHEN place LIKE '%Antarctica%' THEN 'Antarctica' ELSE 'Other' END AS continent,AVG(depth_km) AS avg_depth_km FROM earthquakes GROUP BY continent"
pd.read_sql(query, con=engine)

,continent,avg_depth_km
0,Other,58.164988
1,Africa,9.454506
2,America,16.920136
3,Australia,9.893238
4,Asia,37.010000


### 5. Average magnitude per magnitude type (magType)

In [ ]:
query = "SELECT magnitude_type,AVG(magnitude) AS avg_magnitude FROM earthquakes GROUP BY magnitude_type ORDER BY avg_magnitude DESC"
pd.read_sql(query, con=engine)

,magnitude_type,avg_magnitude
0,mwc,6.150000
1,mwb,5.806667
2,ms_20,5.800000
3,mwp,5.418750
4,mww,5.370369
5,ml(texnet),5.200000
6,mw,5.024887
7,ms_vx,4.900000
8,ml,4.809400
9,mb,4.688584


## Time Analysis

### 6. Year with most earthquakes (FIXED)

In [ ]:
query = "SELECT YEAR(time) AS year,COUNT(*) AS total FROM earthquakes GROUP BY YEAR(time) ORDER BY total DESC LIMIT 1"
pd.read_sql(query, con=engine)

,year,total
0,2021,8959


### 7. Month with highest number of earthquakes

In [ ]:
query = "SELECT MONTH(time) AS month,COUNT(*) AS total FROM earthquakes GROUP BY MONTH(time) ORDER BY total DESC LIMIT 1"
pd.read_sql(query, con=engine)

,month,total
0,8,4314


### 8. Day of week with most earthquakes

In [ ]:
query = "SELECT DAYNAME(time) AS day_of_week,COUNT(*) AS total FROM earthquakes GROUP BY DAYNAME(time) ORDER BY total DESC LIMIT 1"
pd.read_sql(query, con=engine)

,day_of_week,total
0,Friday,6792


### 9. Count of earthquakes per hour

In [ ]:
query = "SELECT HOUR(time) AS hour,COUNT(*) AS total FROM earthquakes GROUP BY HOUR(time) ORDER BY hour"
pd.read_sql(query, con=engine)

,hour,total
0,0,1942
1,1,2023
2,2,1965
3,3,2073
4,4,1984
5,5,1840
6,6,1709
7,7,1848
8,8,1778
9,9,1878


### 10. Most active reporting network (net) 

In [ ]:
query = "SELECT network,COUNT(*) AS total FROM earthquakes GROUP BY network ORDER BY total DESC LIMIT 1"
pd.read_sql(query, con=engine)

,network,total
0,us,45398


## Casualties & Economic Loss

### 11. Top 5 places with highest casualties - column not found

In [ ]:
query = "SELECT place,SUM(casualties) AS total_casualties FROM earthquakes GROUP BY place ORDER BY total_casualties DESC LIMIT 5"
pd.read_sql(query, con=engine)

DatabaseError: Execution failed on sql 'SELECT place,SUM(casualties) AS total_casualties FROM earthquakes GROUP BY place ORDER BY total_casualties DESC LIMIT 5': (mysql.connector.errors.ProgrammingError) 1054 (42S22): Unknown column 'casualties' in 'field list'
[SQL: SELECT place,SUM(casualties) AS total_casualties FROM earthquakes GROUP BY place ORDER BY total_casualties DESC LIMIT 5]
(Background on this error at: https://sqlalche.me/e/20/f405)

### 12. Total estimated economic loss per continent - column not found

In [ ]:
query = "SELECT CASE WHEN place LIKE '%Asia%' THEN 'Asia' WHEN place LIKE '%Europe%' THEN 'Europe' WHEN place LIKE '%Africa%' THEN 'Africa' WHEN place LIKE '%America%' THEN 'America' WHEN place LIKE '%Australia%' THEN 'Australia' ELSE 'Other' END AS continent,SUM(economic_loss) AS total_loss FROM earthquakes GROUP BY continent;"
pd.read_sql(query, con=engine)


DatabaseError: Execution failed on sql 'SELECT CASE WHEN place LIKE '%Asia%' THEN 'Asia' WHEN place LIKE '%Europe%' THEN 'Europe' WHEN place LIKE '%Africa%' THEN 'Africa' WHEN place LIKE '%America%' THEN 'America' WHEN place LIKE '%Australia%' THEN 'Australia' ELSE 'Other' END AS continent,SUM(economic_loss) AS total_loss FROM earthquakes GROUP BY continent;': (mysql.connector.errors.ProgrammingError) 1054 (42S22): Unknown column 'economic_loss' in 'field list'
[SQL: SELECT CASE WHEN place LIKE '%Asia%' THEN 'Asia' WHEN place LIKE '%Europe%' THEN 'Europe' WHEN place LIKE '%Africa%' THEN 'Africa' WHEN place LIKE '%America%' THEN 'America' WHEN place LIKE '%Australia%' THEN 'Australia' ELSE 'Other' END AS continent,SUM(economic_loss) AS total_loss FROM earthquakes GROUP BY continent;]
(Background on this error at: https://sqlalche.me/e/20/f405)

### 13. Average economic loss by alert level - column not found

In [ ]:

pd.read_sql(query, con=engine)

,status,total
0,reviewed,45840


## Event Type & Quality Metrics

### 14. Count of reviewed vs automatic earthquakes (status).

In [ ]:
query = "SELECT status,COUNT(*) AS total FROM earthquakes GROUP BY status"
pd.read_sql(query, con=engine)

,status,total
0,reviewed,45840


### 15. Count by earthquake type (type).

In [ ]:
query = "SELECT event_type,COUNT(*) AS total FROM earthquakes GROUP BY event_type ORDER BY total DESC"
pd.read_sql(query, con=engine)

,event_type,total
0,earthquake,45828
1,volcanic eruption,9
2,landslide,2
3,mine collapse,1


### 16. Number of earthquakes by data type (types).

In [ ]:
query = "SELECT types,COUNT(*) AS total FROM earthquakes GROUP BY types ORDER BY total DESC"
pd.read_sql(query, con=engine)

,types,total
0,",origin,phase-data,",32084
1,",dyfi,origin,phase-data,",4066
2,",dyfi,moment-tensor,origin,phase-data,",1068
3,",earthquake-name,origin,phase-data,",860
4,",moment-tensor,origin,phase-data,",779
...,...,...
423,",dyfi,ground-failure,impact-text,internal-mome...",1
424,",dyfi,earthquake-name,ground-failure,internal-...",1
425,",dyfi,earthquake-name,finite-fault,ground-fail...",1
426,",dyfi,ground-failure,impact-link,losspager,ori...",1


### 17. Average RMS and gap per continent.

In [ ]:
query = "SELECT CASE WHEN place LIKE '%Asia%' THEN 'Asia' WHEN place LIKE '%Europe%' THEN 'Europe' WHEN place LIKE '%Africa%' THEN 'Africa' WHEN place LIKE '%America%' THEN 'America' WHEN place LIKE '%Australia%' THEN 'Australia' WHEN place LIKE '%Antarctica%' THEN 'Antarctica' ELSE 'Other' END AS continent,AVG(rms_error) AS avg_rms,AVG(azimuthal_gap) AS avg_gap FROM earthquakes GROUP BY continent"
pd.read_sql(query, con=engine)

,continent,avg_rms,avg_gap
0,Other,0.733196,91.324337
1,Africa,0.647381,79.529762
2,America,0.832727,114.522727
3,Australia,0.767619,70.119048
4,Asia,0.760000,131.000000


### 18. Events with high station coverage (nst > threshold).

In [ ]:
query = "SELECT id,station_count,magnitude,place,time FROM earthquakes WHERE station_count>100 ORDER BY station_count DESC"
pd.read_sql(query, con=engine)

,id,station_count,magnitude,place,time
0,us6000m12f,619.0,5.4,"11 km W of Anamizu, Japan",2024-01-02 01:17:32
1,us6000qzfl,566.0,5.0,"120 km ENE of Ozernovskiy, Russia",2025-08-09 08:01:41
2,us7000pvtr,475.0,6.8,Macquarie Island region,2025-04-29 14:53:38
3,usd001097k,466.0,5.5,"49 km WNW of San Antonio de los Cobres, Argentina",2023-12-11 18:36:00
4,us7000ilwt,452.0,6.0,North Pacific Ocean,2022-11-02 04:53:13
...,...,...,...,...,...
5590,us6000rgsu,101.0,4.9,"61 km W of Ovalle, Chile",2025-10-13 01:28:07
5591,us6000rn27,101.0,5.1,"256 km N of Tobelo, Indonesia",2025-11-13 03:31:02
5592,us6000rl60,101.0,4.8,"160 km SE of Petropavlovsk-Kamchatsky, Russia",2025-11-03 11:21:32
5593,us7000rlha,101.0,5.5,"23 km N of Wadomari, Japan",2025-12-30 08:12:25


## Tsunamis & Alerts

### 19. Number of tsunamis triggered per year.

In [ ]:
query = "SELECT YEAR(time) AS year,COUNT(*) AS tsunami_count FROM earthquakes WHERE tsunami=1 GROUP BY YEAR(time) ORDER BY year"
pd.read_sql(query, con=engine)

,year,tsunami_count
0,2020,98
1,2021,89
2,2022,100
3,2023,88
4,2024,81
5,2025,106


### 20. Count earthquakes by alert levels (red, orange, etc.). - column not found

In [ ]:
query = "SELECT alert_level,COUNT(*) AS total FROM earthquakes GROUP BY alert_level ORDER BY total DESC"
pd.read_sql(query, con=engine)

DatabaseError: Execution failed on sql 'SELECT alert_level,COUNT(*) AS total FROM earthquakes GROUP BY alert_level ORDER BY total DESC': (mysql.connector.errors.ProgrammingError) 1054 (42S22): Unknown column 'alert_level' in 'field list'
[SQL: SELECT alert_level,COUNT(*) AS total FROM earthquakes GROUP BY alert_level ORDER BY total DESC]
(Background on this error at: https://sqlalche.me/e/20/f405)

## Seismic Pattern & Trends Analysis

### 21.Find the top 5 countries with the highest average magnitude of earthquakes in the past 10 years

In [ ]:
query = "SELECT country,AVG(magnitude) AS avg_magnitude FROM (SELECT CASE WHEN place LIKE '%Japan%' THEN 'Japan' WHEN place LIKE '%Indonesia%' THEN 'Indonesia' WHEN place LIKE '%Chile%' THEN 'Chile' WHEN place LIKE '%Mexico%' THEN 'Mexico' WHEN place LIKE '%China%' THEN 'China' WHEN place LIKE '%India%' THEN 'India' WHEN place LIKE '%USA%' OR place LIKE '%United States%' THEN 'USA' ELSE 'Other' END AS country,magnitude,time FROM earthquakes WHERE time>=DATE_SUB(CURDATE(),INTERVAL 10 YEAR)) t GROUP BY country ORDER BY avg_magnitude DESC LIMIT 5"
pd.read_sql(query, con=engine)

,country,avg_magnitude
0,Mexico,4.867885
1,Chile,4.813471
2,Other,4.807890
3,Indonesia,4.787082
4,Japan,4.776669


### 22. Top 5 countries with highest average magnitude in the past 10 years

In [ ]:
query = "SELECT country,AVG(magnitude) AS avg_magnitude FROM (SELECT CASE WHEN place LIKE '%Japan%' THEN 'Japan' WHEN place LIKE '%Indonesia%' THEN 'Indonesia' WHEN place LIKE '%Chile%' THEN 'Chile' WHEN place LIKE '%Mexico%' THEN 'Mexico' WHEN place LIKE '%USA%' OR place LIKE '%United States%' THEN 'USA' WHEN place LIKE '%China%' THEN 'China' WHEN place LIKE '%India%' THEN 'India' ELSE 'Other' END AS country,magnitude,time FROM earthquakes WHERE time>=DATE_SUB(CURDATE(),INTERVAL 10 YEAR)) t GROUP BY country ORDER BY avg_magnitude DESC LIMIT 5"
pd.read_sql(query, con=engine)

,country,avg_magnitude
0,Mexico,4.867885
1,Chile,4.813471
2,Other,4.807890
3,Indonesia,4.787082
4,Japan,4.776669


### 23. Year-over-year growth rate of total earthquakes globally

In [ ]:
query = "SELECT year,((total-LAG(total) OVER(ORDER BY year))/LAG(total) OVER(ORDER BY year))*100 AS yoy_growth_pct FROM (SELECT YEAR(time) AS year,COUNT(*) AS total FROM earthquakes GROUP BY YEAR(time)) t"
pd.read_sql(query, con=engine)

,year,yoy_growth_pct
0,2020,NaN
1,2021,37.5979
2,2022,-13.3162
3,2023,-1.4808
4,2024,-16.3900
5,2025,33.7502


### 24. Top 3 most seismically active regions (frequency + avg magnitude)

In [ ]:
query = "SELECT region,COUNT(*) AS frequency,AVG(magnitude) AS avg_magnitude,(COUNT(*)*AVG(magnitude)) AS activity_score FROM (SELECT CASE WHEN place LIKE '%Ring of Fire%' OR place LIKE '%Japan%' OR place LIKE '%Indonesia%' OR place LIKE '%Chile%' THEN 'Pacific Ring of Fire' WHEN place LIKE '%Himalaya%' OR place LIKE '%India%' OR place LIKE '%Nepal%' THEN 'Himalayan Belt' WHEN place LIKE '%Mediterranean%' OR place LIKE '%Turkey%' OR place LIKE '%Italy%' THEN 'Mediterranean Belt' ELSE 'Other' END AS region,magnitude FROM earthquakes) t GROUP BY region ORDER BY activity_score DESC LIMIT 3"
pd.read_sql(query, con=engine)

,region,frequency,avg_magnitude,activity_score
0,Other,35067,4.807676,168590.75725
1,Pacific Ring of Fire,8783,4.786667,42041.30000
2,Himalayan Belt,1595,4.776207,7618.05000


## Depth, Location & Distance-Based Analysis.

### 25. For each country, calculate the average depth of earthquakes within ±5° latitude range of the equator.

In [ ]:
query = "SELECT country,AVG(depth_km) AS avg_depth FROM (SELECT CASE WHEN place LIKE '%Japan%' THEN 'Japan' WHEN place LIKE '%Indonesia%' THEN 'Indonesia' WHEN place LIKE '%Chile%' THEN 'Chile' WHEN place LIKE '%Mexico%' THEN 'Mexico' WHEN place LIKE '%USA%' OR place LIKE '%United States%' THEN 'USA' ELSE 'Other' END AS country,depth_km,latitude FROM earthquakes WHERE latitude BETWEEN -5 AND 5) t GROUP BY country"
pd.read_sql(query, con=engine)

,country,avg_depth
0,Other,41.471342
1,Indonesia,51.810524
2,USA,10.000000


### 26. Identify countries having the highest ratio of shallow to deep earthquakes.

In [ ]:
query = "SELECT country,SUM(depth_km<70)/NULLIF(SUM(depth_km>300),0) AS shallow_to_deep_ratio FROM (SELECT CASE WHEN place LIKE '%Japan%' THEN 'Japan' WHEN place LIKE '%Indonesia%' THEN 'Indonesia' WHEN place LIKE '%Chile%' THEN 'Chile' WHEN place LIKE '%Mexico%' THEN 'Mexico' WHEN place LIKE '%USA%' OR place LIKE '%United States%' THEN 'USA' ELSE 'Other' END AS country,depth_km FROM earthquakes) t GROUP BY country ORDER BY shallow_to_deep_ratio DESC"
pd.read_sql(query, con=engine)

,country,shallow_to_deep_ratio
0,Indonesia,36.6279
1,Japan,29.8660
2,Other,17.4474
3,Mexico,NaN
4,Chile,NaN
5,USA,NaN


### 27. Find the average magnitude difference between earthquakes with tsunami alerts and those without.

In [ ]:
query = "SELECT (SELECT AVG(magnitude) FROM earthquakes WHERE tsunami=1)-(SELECT AVG(magnitude) FROM earthquakes WHERE tsunami=0) AS avg_magnitude_difference"
pd.read_sql(query, con=engine)

,avg_magnitude_difference
0,1.038199


### 28. Using the gap and rms columns, identify events with the lowest data reliability (highest average error margins).

In [ ]:
query = "SELECT id,place,((azimuthal_gap+rms_error)/2) AS avg_error FROM earthquakes ORDER BY avg_error DESC LIMIT 10"
pd.read_sql(query, con=engine)

,id,place,avg_error
0,pr2020283000,"85 km ENE of Saint Croix, U.S. Virgin Islands",174.245
1,nc73351825,"73km W of Petrolia, CA",160.595
2,pr2020331007,"105 km N of Charlotte Amalie, U.S. Virgin Islands",158.690
3,us6000a6gy,South Sandwich Islands region,145.770
4,us7000dfhu,"248 km ENE of Gisborne, New Zealand",145.450
5,us7000emi2,Chagos Archipelago region,141.975
6,nc73541781,"76km W of Petrolia, CA",141.140
7,pr2020035000,"61 km NE of Miches, Dominican Republic",140.940
8,us6000lp5x,west of the Galapagos Islands,140.800
9,us7000dfgb,"294 km ENE of Gisborne, New Zealand",140.315


### 29. Find pairs of consecutive earthquakes (by time) that occurred within 50 km of each other and within 1 hour.

In [ ]:
query = "SELECT region,COUNT(*) AS deep_event_count FROM (SELECT CASE WHEN place LIKE '%Japan%' OR place LIKE '%Indonesia%' OR place LIKE '%Chile%' THEN 'Pacific Ring of Fire' WHEN place LIKE '%India%' OR place LIKE '%Nepal%' THEN 'Himalayan Belt' WHEN place LIKE '%Turkey%' OR place LIKE '%Italy%' THEN 'Mediterranean Belt' ELSE 'Other' END AS region,depth_km FROM earthquakes WHERE depth_km>300) t GROUP BY region ORDER BY deep_event_count DESC"
pd.read_sql(query, con=engine)

,region,deep_event_count
0,Other,1681
1,Pacific Ring of Fire,183
2,Mediterranean Belt,2


### 30. Determine the regions with the highest frequency of deep-focus earthquakes (depth > 300 km).

In [ ]:
query = "SELECT region,COUNT(*) AS deep_event_count FROM (SELECT CASE WHEN place LIKE '%Japan%' OR place LIKE '%Indonesia%' OR place LIKE '%Chile%' THEN 'Pacific Ring of Fire' WHEN place LIKE '%India%' OR place LIKE '%Nepal%' THEN 'Himalayan Belt' WHEN place LIKE '%Turkey%' OR place LIKE '%Italy%' THEN 'Mediterranean Belt' ELSE 'Other' END AS region,depth_km FROM earthquakes WHERE depth_km>300) t GROUP BY region ORDER BY deep_event_count DESC"
pd.read_sql(query, con=engine)

,region,deep_event_count
0,Other,1681
1,Pacific Ring of Fire,183
2,Mediterranean Belt,2


In [ ]:
df.place.head(2)

0       6 km NNE of Sivrice, Turkey
1    21 km ENE of San Bartolo, Peru
Name: place, dtype: str